
# Práctica 1: Modelo Poblacional Logístico (Parte 2)

**Enunciado general.**  
En este cuaderno estudiaremos un modelo poblacional **logístico** aplicado a un caso real: una población de **conejos** $N(t)$ que vive en un campo cuya **capacidad de carga** $K$ viene determinada por la **capacidad de producir zanahorias**. Analizaremos y compararemos la **solución analítica** y la **solución numérica**, mostraremos **gráficas de $N(t)$** y el **campo de crecimiento** (slope field), y añadiremos **interactividad** con los parámetros del sistema y la condición inicial. En cada ejercicio encontrarás:
- Un **enunciado** con lo que se pide.
- Bloques de **preguntas y reflexión**.
- Debes generar un nuevo bloque con el código que se pide en cada ejercicio.

## Ejemplo de funciones para gráficas

A continuación se muestra el código, organizado en funciones, para poder mostrar diagramas de fase y campos de crecimiento:


In [ ]:

# =========================================
# Importaciones y utilidades comunes
# =========================================
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

try:
    from ipywidgets import interact, FloatSlider, IntSlider
    import ipywidgets as widgets
except Exception as e:
    print("Si ipywidgets no está disponible, instala con: pip install ipywidgets y reinicia el entorno.")

plt.rcParams["figure.figsize"] = (7, 4)
plt.rcParams["axes.grid"] = True

def logistic_rhs(t, N, r, K):
    return r * N * (1 - N / K)

def logistic_analytical(t, r, K, N0):
    t = np.asarray(t)
    if K <= 0:
        return np.full_like(t, np.nan, dtype=float)
    if N0 == 0.0:
        return np.zeros_like(t, dtype=float)
    A = (K - N0) / N0
    return K / (1.0 + A * np.exp(-r * t))

def growth_field_logistic(r, K, N0=None, N_max=None, n_N=200):
    # Graficar dN/dt como función de N, con punto en la CI
    if N_max is None:
        N_max = K * 1.5 if K > 0 else 10.0
    N_values = np.linspace(0, N_max, n_N)
    dNdt = logistic_rhs(0, N_values, r, K)

    plt.figure()
    plt.plot(N_values, dNdt, lw=2, label="dN/dt vs N")
    plt.axvline(K, linestyle='--', label='Cap. de carga K')
    if N0 is not None:
        plt.scatter([N0], [logistic_rhs(0, N0, r, K)], c='red', s=60, label="C.I.")
    plt.xlabel("N (conejos)")
    plt.ylabel("dN/dt")
    plt.title("Campo de crecimiento: dN/dt en función de N")
    plt.legend()
    plt.show()



## Ejercicio 1. Modelo poblacional logístico (conejos y zanahorias): analítico vs. numérico

**Contexto real.**  
Supón una población de **conejos** $N(t)$ en un campo. La capacidad de carga $K$ depende de cuántas **zanahorias** puede producir el campo (alimento disponible). El crecimiento se modela como:
$$
\frac{dN}{dt} = r\,N\Bigl(1 - \frac{N}{K}\Bigr), \quad r>0,\; K>0.
$$
- $r$: tasa intrínseca de crecimiento de la población (ej., fecundidad neta).  
- $K$: capacidad de carga (máximo sostenible por el entorno — aquí: zanahorias disponibles).

**Tareas.**
1. Compara la **solución analítica** $N(t)$ con la **solución numérica** (misma C.I. $N(0)=N_0$).  
2. Dibuja:
   - La evolución $N(t)$, marcando con un **punto rojo** la **condición inicial**.
   - El **campo de crecimiento** (slope field) del modelo.

**A continuación se proporciona la solución:**


In [ ]:

# Parámetros por defecto
r = 0.8
K = 50.0
N0 = 5.0
t_max = 20.0
n_pts = 800
t = np.linspace(0, t_max, n_pts)

# Solución analítica
N_anal = logistic_analytical(t, r, K, N0)

# Solución numérica
def rhs(t, y): return logistic_rhs(t, y, r, K)
sol = solve_ivp(lambda _t, _y: rhs(_t, _y), [0, t_max], [N0], t_eval=t, rtol=1e-9, atol=1e-12)
N_num = sol.y[0]

# Gráfica N(t) analítica vs numérica
plt.figure()
plt.plot(t, N_anal, label="N(t) analítica", lw=2)
plt.plot(t, N_num, '--', label="N(t) numérica", lw=2)
plt.scatter([0], [N0], s=60, c='red', label="C.I.")
plt.xlabel("t")
plt.ylabel("N(t) (conejos)")
plt.title("Modelo logístico: N(t) analítica vs numérica")
plt.legend()
plt.show()

# Campo de crecimiento: dN/dt vs N
N_max_plot = max(1.5*K, 1.25*N0 + 1.0)
growth_field_logistic(r, K, N0=N0, N_max=N_max_plot)



> **Preguntas (responder en texto):**
> 1. Clasifica el sistema según:
>    - Relación entrada–salida  
>    - Tipo de incertidumbre  
>    - Naturaleza de tiempo  
>    - Dependencia temporal
>
> 2. Interpreta **razonadamente** el comportamiento observado para $N(t)$: ¿Cómo influye el valor de $r$?



## Ejercicio 2. Interacción con $r$, $K$ y la C.I.

**Enunciado.** Añade interactividad para variar $r$, $K$, $N_0$ y el tiempo de simulación $t_{\max}$. Debe actualizarse la comparación **analítica vs. numérica** de $N(t)$ y el **campo de crecimiento**.

> **Ejecuta y analiza estos tres casos de condiciones iniciales:**
> 1. $N_0 < K$ pero **cercano** a $K$.  
> 2. $N_0 > K$ pero **cercano** a $K$.  
> 3. $N_0 = K$.  
>
> **Explica y razona** qué ocurre en cada caso al simular.
